<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MAGISTRAL_AGENT_DEMO_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://mistral.ai/news/agents-api

https://console.mistral.ai/usage

In [ ]:
!pip install mistralai --quiet

In [2]:
from IPython import get_ipython
from IPython.display import display
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral

In [3]:
from mistralai import Mistral


from google.colab import userdata
api_key = userdata.get('MISTRAL_API_KEY')

client = Mistral(api_key=api_key)


client = Mistral(api_key=api_key)  # Use MistralClient instead of Mistral
model_list = client.models.list()

for model in model_list.data:
    print(model.id)
    #print(model.created)
    #print(model.owned_by)

mistral-medium-2505
mistral-large-latest
mistral-medium-2508
mistral-medium-latest
mistral-medium
ministral-3b-2410
ministral-3b-latest
ministral-8b-2410
ministral-8b-latest
open-mistral-7b
mistral-tiny
mistral-tiny-2312
open-mistral-nemo
open-mistral-nemo-2407
mistral-tiny-2407
mistral-tiny-latest
open-mixtral-8x7b
mistral-small
mistral-small-2312
open-mixtral-8x22b
open-mixtral-8x22b-2404
mistral-small-2409
mistral-large-2407
mistral-large-2411
pixtral-large-2411
pixtral-large-latest
mistral-large-pixtral-2411
codestral-2501
codestral-2412
codestral-2411-rc5
codestral-2508
codestral-latest
devstral-small-2505
devstral-small-2507
devstral-small-latest
devstral-medium-2507
devstral-medium-latest
pixtral-12b-2409
pixtral-12b
pixtral-12b-latest
mistral-small-2501
mistral-small-2503
mistral-small-2506
mistral-small-latest
mistral-saba-2502
mistral-saba-latest
magistral-medium-2506
magistral-medium-2507
magistral-small-2506
magistral-small-2507
magistral-medium-2509
magistral-medium-latest

In [2]:
import os
import json
from pydantic import BaseModel
from mistralai import Mistral
from mistralai.models import Function, Tool  # Adjust if these are in a different module
from google.colab import userdata

# Retrieve API key from Colab secrets
api_key = userdata.get('MISTRAL_API_KEY')
if not api_key:
    print("Error: MISTRAL_API_KEY not found in Colab secrets.")
    print("Please set your Mistral API key in Colab's secrets management.")
    exit()

# Initialize Mistral client
client = Mistral(api_key=api_key)

# --- Agent Definitions ---
class CalcResult(BaseModel):
    reasoning: str
    result: str

print("Creating AI agents...")

finance_agent = client.beta.agents.create(
    model="magistral-medium-latest",
    description="Agent used to answer financial related requests",
    name="finance-agent",
)
web_search_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent that can search online for any information if needed",
    name="websearch-agent",
    tools=[{"type": "web_search"}],
)
ecb_interest_rate_agent = client.beta.agents.create(
    model="magistral-medium-latest",
    description="Can find the current interest rate of the European central bank",
    name="ecb-interest-rate-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_european_central_bank_interest_rate",
                "description": "Retrieve the real interest rate of European central bank.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {"type": "string", "description": "Date in YYYY-MM-DD format"},
                    },
                    "required": ["date"]
                },
            },
        },
    ],
)
graph_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="graph-drawing-agent",
    description="Agent used to create graphs using the code interpreter tool.",
    instructions="Use the code interpreter tool when you have to draw a graph.",
    tools=[{"type": "code_interpreter"}]
)
calculator_agent = client.beta.agents.create(
    model="magistral-medium-latest",
    name="calculator-agent",
    description="Agent used to make detailed calculations",
    instructions="When doing calculations explain step by step what you are doing.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "calc_result",
                "schema": CalcResult.model_json_schema(),
            }
        }
    }
)

# --- Custom function for flight search (MOCK) ---
def search_flights(origin: str, destination: str, departure_date: str, return_date: str = None):
    """
    MOCK: This function simulates searching for available flights.
    It returns dummy data based on specific input parameters.
    """
    print(f"\n[DEBUG] MOCK CALL: search_flights with origin='{origin}', destination='{destination}', departure_date='{departure_date}', return_date='{return_date}'")

    origin_norm = origin.upper() if origin else ""
    destination_norm = destination.upper() if destination else ""

    if origin_norm == "YUL" and destination_norm == "JFK" and departure_date == "2025-10-01":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "AC700", "airline": "Air Canada", "departure_time": "10:00 AM", "arrival_time": "11:30 AM", "price": "$250 CAD"},
                {"flight_number": "DL123", "airline": "Delta Airlines", "departure_time": "11:00 AM", "arrival_time": "12:45 PM", "price": "$280 CAD"}
            ]
        }
    elif origin_norm == "LAX" and destination_norm == "SFO" and departure_date == "2025-10-15":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "UA456", "airline": "United Airlines", "departure_time": "09:00 AM", "arrival_time": "10:15 AM", "price": "$120 USD"},
            ]
        }
    else:
        return {"flights_found": False, "details": "No flights found for the specified criteria."}

# --- Flight Planning Agent Definition ---
flight_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting with flight planning requests, searching for flights and providing relevant information. Use IATA airport codes for origin and destination if possible. For example, Montreal is YUL and New York is JFK or LGA.",
    name="flight-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_flights",
                "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                        "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                        "departure_date": {"type": "string", "description": "Departure date in YYYY-MM-DD format"},
                        "return_date": {"type": "string", "description": "Return date in YYYY-MM-DD format (optional)"}
                    },
                    "required": ["origin", "destination", "departure_date"]
                }
            }
        },
        {"type": "web_search"}
    ]
)

print(f"\nFlight Planning Agent '{flight_planning_agent.name}' created with ID: {flight_planning_agent.id}")
tool_names_list = []
for tool in flight_planning_agent.tools:
    if tool.type == 'function':
        tool_names_list.append(tool.function.name)
    else:
        tool_names_list.append(tool.type)
print(f"Tools available to flight_planning_agent: {tool_names_list}")

# --- Test Case Execution for Flight Planning Agent ---
print("\n--- Executing Test Case for the Flight Planning Agent (via chat completions) ---")
print("This simulates a multi-turn conversation where your code acts as the tool executor.")

conversation_history = []
user_query = "Find me a flight from Montreal to New York on October 1st, 2025."
print(f"\nUser: {user_query}")
conversation_history.append({"role": "user", "content": user_query})

try:
    search_flights_function = Function(
        name="search_flights",
        description="Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
        parameters={
            "type": "object",
            "properties": {
                "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                "departure_date": {"type": "string", "description": "Departure date in YYYY-MM-DD format"},
                "return_date": {"type": "string", "description": "Return date in YYYY-MM-DD format (optional)"}
            },
            "required": ["origin", "destination", "departure_date"]
        }
    )

    # Only include search_flights tool to avoid validation error
    api_call_tools_list = [
        Tool(function=search_flights_function)
    ]

    print("[DEBUG] Sending initial user query to the agent...")
    response_turn1 = client.chat.complete(
        model=flight_planning_agent.model,
        messages=conversation_history,
        tools=api_call_tools_list,
    )

    assistant_message_turn1 = response_turn1.choices[0].message
    conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1.__dict__)

    if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
        print("\nAgent proposed tool calls (Turn 1):")
        for tool_call in assistant_message_turn1.tool_calls:
            print(f"  Tool Name: {tool_call.function.name}")
            print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")

            tool_output_content = None
            if tool_call.function.name == "search_flights":
                try:
                    args = json.loads(tool_call.function.arguments)
                    tool_output = search_flights(**args)
                    tool_output_content = json.dumps(tool_output)
                    print(f"  [DEBUG] Local MOCK search_flights executed. Output: {tool_output_content}")
                except json.JSONDecodeError as e:
                    print(f"  [ERROR] Failed to parse tool arguments: {e}")
                    tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                except Exception as e:
                    print(f"  [ERROR] Error executing local mock search_flights: {e}")
                    tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})

            else:
                print(f"  [DEBUG] Unhandled tool call: {tool_call.function.name}")
                tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

            conversation_history.append(
                {
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": tool_output_content,
                    "tool_call_id": tool_call.id
                }
            )
            print(f"  [DEBUG] Tool output for '{tool_call.function.name}' added to history with tool_call_id: {tool_call.id}.")

        print("\n[DEBUG] Sending conversation history with tool outputs back for final response...")
        final_response = client.chat.complete(
            model=flight_planning_agent.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        final_assistant_message = final_response.choices[0].message
        print("\nAgent's Final Response:")
        print(final_assistant_message.content)
        conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message.__dict__)
    else:
        print("\nAgent's initial response (no tool calls proposed):")
        print(assistant_message_turn1.content)

except Exception as e:
    print(f"\nAn error occurred during agent interaction: {e}")
    print("Please check your API key, model availability, network connection, or SDK version.")
    print("Run `!pip install --upgrade mistralai` in Colab to ensure the latest SDK version.")
    print("If issues persist, restart the Colab runtime and try again.")

print("\n--- Test case execution complete. ---")

Creating AI agents...

Flight Planning Agent 'flight-planning-agent' created with ID: ag_019963d4c03b76ff865254ab272607aa
Tools available to flight_planning_agent: ['search_flights', 'web_search']

--- Executing Test Case for the Flight Planning Agent (via chat completions) ---
This simulates a multi-turn conversation where your code acts as the tool executor.

User: Find me a flight from Montreal to New York on October 1st, 2025.
[DEBUG] Sending initial user query to the agent...

Agent proposed tool calls (Turn 1):
  Tool Name: search_flights
  Tool Arguments (JSON string): {"origin": "YUL", "destination": "JFK", "departure_date": "2025-10-01"}

[DEBUG] MOCK CALL: search_flights with origin='YUL', destination='JFK', departure_date='2025-10-01', return_date='None'
  [DEBUG] Local MOCK search_flights executed. Output: {"flights_found": true, "details": [{"flight_number": "AC700", "airline": "Air Canada", "departure_time": "10:00 AM", "arrival_time": "11:30 AM", "price": "$250 CAD"}, {"